## Movie Review Sentiment Classification

### Following instructions from Ch.8 and Ch.9 of "Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn and TensorFlow2" Third Edition by Sebastian Raschka & Vahid Mirjalili

### Using dataset provided by Stanford, at : https://ai.stanford.edu/~amaas/data/sentiment/ (not provided in repo for size purposes, just put  'acllmdb' extract-folder in same folder of clone)

### Carter Roberts
### For Loyola University New Orleans
### March 17, 2025

In [2]:
# some of the notes are just for me and my forward-going comprehension, 
# but you should read them just in case
import pyprind as pyp
import pandas as pd
import os

# this will need to be adjusted to what your path for aclImdb dataset is
basepath = 'C:\\Users\\Carter\\Documents\\Github\\maLearningMovieClassification\\aclImdb'

# establishing diction for columns
labels = {'pos': 1, 'neg': 0}
# 50,000 step progress bar & empty dataframe
pbar = pyp.ProgBar(50000)
df = pd.DataFrame()
# sift through all of those two folders with those two review results
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        # fix up a variable 'path' for next loop
        path = os.path.join(basepath, s, l)
        # use 'path' to loop through all files in the current directory within 'test' or 'train'
        for file in sorted(os.listdir(path)):
            # 'infile' now defines the current total path which was assembled above
            with open(os.path.join(path, file), 'r', encoding = 'utf-8') as infile:
                # read, from the file, the written data, into these holders, 'txt'
                txt = infile.read()
                # put these into dataframe row instances
                new_row = pd.DataFrame([[txt, labels[l]]])
                # put these rows into the df dataframe
                df = pd.concat([df, new_row], ignore_index=True)
                # iterate a step of the progress bar
                pbar.update()
# define the columns of this newly-filled dataframe
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:16:50


In [3]:
# let's reshuffle the data and make it a csv file
import numpy as np

# pick a seed for randomization, in this case just that for 0
np.random.seed(0)
# do the actual randomization
df = df.reindex(np.random.permutation(df.index))
# generate a csv using df
df.to_csv('movie_data.csv', index = False, encoding = 'utf-8')

In [4]:
# regenerate df based on that csv and load the first 3 examples
df = pd.read_csv('./movie_data.csv', encoding = 'utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [5]:
# sanity check: size should be 50,000 x 2
df.shape

(50000, 2)

In [6]:
# display last 50 characters from first document in reshuffled movie review dataset, to show why 
# cleaning up the vocab is necessary for bag-of-words, which will be the chosen method to fix out
# text for the model
df.loc[0, 'review'] [-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

We don't need all that HTML nonsense. Stuff like ":)" can still be useful, though. Let's have it keep emoticons with Python's 'regular expression' library

In [7]:
import re
# method to process out unwanted characters / markup / text and distinguish emoticons
def preprocessor(text):
    # redefine entered text as itself, with the weird markup marks parameterized removed
    text = re.sub('<[^>]*>', '', text)
    # define 'emoticons' as the list of all places where these text-emoticons are
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # redefine entered text again as itself, all lowercase, without \W hypertext and with emoticons if any
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:9: SyntaxWarning: invalid escape sequence '\W'
<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:9: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Carter\AppData\Local\Temp\ipykernel_6508\2402601614.py:7: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
C:\Users\Carter\AppData\Local\Temp\ipykernel_6508\2402601614.py:9: SyntaxWarning: invalid escape sequence '\W'
  text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))


In [8]:
# testing preprocessor function (1)
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [9]:
# testing preprocessor function (2)
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [10]:
# for actual use now, apply the preprocessor function to the review texts
df['review'] = df['review'].apply(preprocessor)

In [11]:
# demonstrating use of text-split tokenizing
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [12]:
# furtheriing this demonstration with the Porter stemming algorithm, 
# which is somewhat superfluous to this but super interesting so I'm doing it
from nltk.stem.porter import PorterStemmer
# define a porter stemmer variable
porter = PorterStemmer()
# method for porter stemming
def tokenizer_porter(text):
    # use stem method of porter stemmer variable for each word in the tokenized text
    return [porter.stem(word) for word in text.split()]
# example
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [13]:
# now, stop-word removal, which is the final step in this tangent
import nltk
# get the set of 127 stop-words from nltk
nltk.download('stopwords')
# import them
from nltk.corpus import stopwords

# hold the stopwords here
stop = stopwords.words('english')
# removes superfluous words that are in stopwords collection!
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Carter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

In [14]:
# finally, return to the df dataset to separate it into training and testing data
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [15]:
# find optimal parameters set for logistic regression using a 5-fold cross-validation achieved with GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None, lowercase = False, preprocessor = None)
# to be honest i don't really get this but I really want to
param_grid = [{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [stop, None], 'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]},
               {'vect__ngram_range': [(1, 1)], 'vect__stop_words': [stop, None], 'vect__tokenizer': [tokenizer, tokenizer_porter],
                'vect__use_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}]
# apply logistic regression to the TfidfVectorizer variable
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state = 0, solver = 'liblinear'))])
# initializes grid search for that logistic regression with the pre-defined parameter grid, for accuracy
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring = 'accuracy', cv = 5, verbose = 2, n_jobs = 1)
# apply this to the training set
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Users\Carter\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ou

c:\Users\Carter\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ou

...

[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x000002696DC66E80>, vect__use_idf=False; total time= 2.1min


c:\Users\Carter\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x000002696DC66E80>, vect__use_idf=False; total time= 2.1min


c:\Users\Carter\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['a', 'about', 'above', 'after',
                                                'again', 'against', 'ain',
                                                'all', 'am', 'an', 'and', 'a...
                                                'again', 'against', 'ain',
                                                'all', 'am', 'an', 'and', 'any',
                                                'are', 'aren', "aren't", 'as',
                                                'at', 'be', 'because', 'been',
                                                'before', 'being', 'below',
                                                'between', 'both', 'but', 'by',
                                                'can', 'couldn', "couldn't", ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x000002696DC66A20>,
                                              <function tokenizer_porter at 0x000002696DC66E80>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [16]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000002696DC66A20>} 


In [17]:
# use this best model that took FIVE HOURS (textbook said "FOURTY FIVE MINUTES") to get accuracy
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

CV Accuracy: 0.897
Test Accuracy: 0.899


That means the model has about 90% Accuracy.

Now let's try out-of-core learning.

In [20]:
# preprocessing method that also tokenizes 
def preproc_tokenize(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

<>:4: SyntaxWarning: invalid escape sequence '\)'
<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:4: SyntaxWarning: invalid escape sequence '\)'
<>:5: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Carter\AppData\Local\Temp\ipykernel_6508\3443950982.py:4: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
C:\Users\Carter\AppData\Local\Temp\ipykernel_6508\3443950982.py:5: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')


In [ ]:
# method that reads in and returns one document at a time
def stream_docs(path):
    # open a csv path-file for reading in utf-8 format
    with open(path, 'r', encoding = 'utf-8') as csv:
        # skip the header
        next(csv)
        for line in csv:
            # separate out text and label vars based on where they'd be as columns in csv format
            text, label = line[:-3], int(line[-2])
            # gives back text & label var values
            yield text, label

In [22]:
# test this out
next(stream_docs(path = 'movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [24]:
# method that takes stream_docs and returns number of docs specified by parameter
def get_minibatch(doc_stream, size):
    # define docs and y as empty lists
    docs, y = [], []
    # test the search
    try:
        # go for as many docs as size parameter
        for _ in range(size):
            # fill up text and label based on doc_stream parameter
            text, label = next(doc_stream)
            # fill up next entry of docs and y with text and label respectively
            docs.append(text)
            y.append(label)
    # when there's no more left
    except StopIteration:
        # there's nothing left! you get nothing!
        return None, None
    # normal return-case, when there's anything
    return docs, y

In [27]:
# vectorize text with 32-bit murmurhash3 function in sklearn using all of these
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

# do the hashingvectorizer/murmurhash3, use preprocessor+tokenizer method
vect = HashingVectorizer(decode_error = 'ignore', n_features = 2**21, preprocessor = None, tokenizer = preproc_tokenize)
# set variable for linear classifier, set to logistic
clf = SGDClassifier(loss = 'log_loss', random_state = 1)
# read in the documents using previously made method for specifically the movie set
doc_str = stream_docs(path = 'movie_data.csv')

In [28]:
# start the out-of-core listening here!
# progress bar & holder numpy array for classification
pbar = pyp.ProgBar(45)
cls = np.array([0, 1])

# for all 45 steps,
for _ in range(45):
    # get 1000 docs from the read-in doc_str and put the docs & y into X_train & y_train respectively
    X_train, y_train = get_minibatch(doc_str, size = 1000)
    # if you got nothing, go home
    if not X_train:
        break
    # apply the HashingVectorizer transformation to X_train 
    X_train = vect.transform(X_train)
    # fit y_train and hashed X_train to linear classifier with the holder numpy array
    clf.partial_fit(X_train, y_train, classes = cls)
    # my little tin soldier
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:01


In [29]:
# get 5000 docs from the read-in doc_str and put the docs & y into X_test & y_test respectively
X_test, y_test = get_minibatch(doc_str, size = 5000)
# apply the HashingVectorizer transformation to X_test
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


That means the model has about 87% Accuracy. Only slightly lower than the previous way, and much, MUCH faster!

In [30]:
# add this whole 'test-set' thing to the model
clf = clf.partial_fit(X_test, y_test)

## Chapter 9 Material

In [31]:
# use pickle to keep model persistent
import pickle
import os

# designate directory, 'SentimentClassification', and subdirectory 'pkl_objects' for pickle
dest = os.path.join('SentimentClassification', 'pkl_objects')
# create it if it doesnt exist
if not os.path.exists(dest):
    os.makedirs(dest)
# use 'dump' method to serialize both the stop-word set and the model
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol = 4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol = 4)